# 대구 날씨 분석

- 기간: 1904.01 ~ 2017.06
- 출처: [기상자료개방포털](https://data.kma.go.kr)
- dgWeahter: 원본
- dgWeather(utf-8): 0~7행 삭제, 누락값 제거, utf-8 인코딩

In [ ]:
## install module
# %pip install jupyter notebook -U
# %pip install pandas
# %pip install matplotlib
# %pip list

In [ ]:
## linux command
# 현재 위치, 현재 위치 안의 자료 리스트
# %pwd
# %ls

In [ ]:
# 운영체제 플랫폼에 따른 한글 폰트 설정
import platform
import matplotlib.pyplot as plt
import matplotlib.font_manager as fm


system_name = platform.system()
if system_name == 'Windows':  # Windows 운영체제
	print('Windows OS')
	plt.rc('font', family='Malgun Gothic')
elif system_name == 'Darwin':  # Mac OS
	print('Mac OS')
	plt.rc('font', family='AppleGothic')
elif system_name == 'Linux':  # Linux
	print('Linux OS')
	path = '/usr/share/fonts/truetype/nanum/NanumMyeongjo.ttf'
	font_name = fm.FontProperties(fname=path, size=12)
	plt.rc('font', family=font_name)
else:
	print("will be support")

---
# 가장 더웠던 날

1. csv로 찾는 방법
2. numpy로 찾는 방법

In [ ]:
import csv


f = open('../data/dgWeather.csv', mode='r', encoding='utf-8')
data = csv.reader(f, delimiter=',')
header = next(data)  # 컬럼 첫번째 줄 header에 담기

max_temp = -999  # 최저값 미리 설정
max_date = ''  # 빈 줄이 아닐 경우를 의미

# 최대값 구하는 알고리즘
for row in data:
    if row[-1] == '':
        row[-1] = -999
    row[-1] = float(row[-1])
    
    if(max_temp < row[-1]):
        max_temp = row[-1]
        max_date = row[0]
f.close()

print(f'대구 기온이 가장 높았던 날은 {max_date}로 {max_temp}도 였습니다.')

In [ ]:
import csv
import numpy as np


f = open('../data/dgWeather.csv', mode='r', encoding='utf-8')
data = csv.reader(f, delimiter=',')
header = next(data)

max_temp = -999
max_date = ''
max_temp_list = []
max_temp_date = []

for row in data:
    if row[-1] == '':
        row[-1] = -999
    max_temp_list.append(row[-1])
    max_temp_date.append(row[0])
f.close()

max_temp_array = np.array(max_temp_list)  # 리스트를 ndarray 타입으로 변경
max_temp_array = max_temp_array.astype(float)  # 문자열 타입을 float으로 변경
max_date_array = np.array(max_temp_date)  # 리스트를 ndarray 타입으로 변경

max_temp = max_temp_array.max()  # 최고온도값
index = max_temp_array.argmax()  # 최대값의 index 리턴
max_date = max_date_array[index]

print(f'대구 기온이 가장 높았던 날은 {max_date}로 {max_temp}도 였습니다.')

---
## 8월 기온 histogram으로 보기

In [ ]:
import csv
import matplotlib.pyplot as plt


f = open('../data/dgWeather.csv', mode='r', encoding='utf-8')
data = csv.reader(f)
header = next(data)

aug = []
for row in data:
    month = row[0].split('.')[1]
    if row[-1] != '':
        if month == '08':
            aug.append(float(row[-1]))
f.close()

plt.hist(aug, bins=100, color='tomato')
plt.xlabel('Temerature')
plt.ylabel('Counts')
plt.show()

---
## 1월과 8월의 기온 분포

In [ ]:
import csv
import matplotlib.pyplot as plt
import pandas as pd


f = open('../data/dgWeather.csv', mode='r', encoding='utf-8')
data = csv.reader(f)
header = next(data)

aug = []
jan = []
for row in data:
    month = row[0].split('-')[1]
    if row[-1] != '':
        if month == '01':
            aug.append(float(row[-1]))
        if month == '08':
            jan.append(float(row[-1]))
f.close()

plt.hist(jan, bins=100, color='green', label='Jan')
plt.hist(aug, bins=100, color='tomato', label='Aug')
plt.xlabel('Temerature')
plt.rc('axes', unicode_minus=False)
plt.legend()
plt.show()

---
## 특정 날짜의 최고 기온 찾기

In [ ]:
import csv
import matplotlib.pyplot as plt


def draw_graph_on_date(month, day):
	# 1월 1일을 01 01이 아닌 1 1로 입력했을 경우
	if not '0' in month and '0'  not in day:
		month = '0' + month
		day = '0' + day
		
	f = open('../data/dgWeather.csv', mode='r', encoding='utf-8')
	data = csv.reader(f)
	next(data)
	
	result = []
	for row in data:
		if row[-1] != '':
			date_string = row[0].split('-')
			if date_string[1] == month and date_string[2] == day:
				result.append(float(row[-1]))
	f.close()

	plt.figure(figsize=(15,2))
	plt.plot(result, 'royalblue')
	plt.ylabel('온도')
	plt.show()

# month, date = input('날짜(월 일) 입력 ').split()
month, date = '6', '6'
draw_graph_on_date(month, date)

---
# pandas로 기온 데이터 분석하기

In [ ]:
import pandas as pd


dg_weather = pd.read_csv('../data/dgWeather.csv')
dg_weather.columns=['날짜', '지점', '평균기온', '최저기온', '최고기온']
dg_weather['날짜'] = pd.to_datetime(dg_weather['날짜'], format='%Y-%m-%d')

# count()는 누락값을 제외한 정상값의 갯수만 카운트
# 전체 행에서 정상값 갯수 빼기
dg_weather.shape[0]- dg_weather.count()

# 누락값 제거
dg_weather = dg_weather.dropna(axis=0)

# 누락값 제거된 최종 데이터를 csv파일로 저장
dg_weather.to_csv('../data/dgWeahter(utf-8).csv', index=False, mode='w', encoding='utf-8')

---
## 2021년 7월 최고기온 평균값

In [ ]:
year_data = dg_weather[dg_weather['날짜'].dt.year==2021]
month_data = year_data[year_data['날짜'].dt.month==7]